In [1]:
from Simulation_PyAPDL import simulation_PyAPDL
from scipy.integrate import simps
import matplotlib.pyplot as plt
import scipy.io as sio
import numpy as np
import json
import os


def get_MAC(mode_shape_model,mode_shape_exp):
    mode_shape_model = mode_shape_model.reshape(1, -1)
    mode_shape_exp = mode_shape_exp.reshape(1, -1)
    return np.power(np.dot(mode_shape_exp,np.transpose(mode_shape_model)),2)[0][0]/(np.dot(mode_shape_model,np.transpose(mode_shape_model))[0][0]*np.dot(mode_shape_exp,np.transpose(mode_shape_exp)))[0][0]

def fit_gaussian_kernel(f,f_n,zeta):
    # Compute the standard deviation (sigma) based on the damping ratio
    band_width_ratio = 1.1
    sigma = band_width_ratio*(zeta * f_n) / np.sqrt(2 * np.log(2))
    return np.exp(-0.5 * ((f - f_n) / sigma)**2)

def find_peaks_SDOFsup(num_peak,natrual_freq_arr,frf_freq_arr,targer_FRF,zeta):
    frf_freq_arr = frf_freq_arr.reshape(1,-1)[0]
    targer_FRF = targer_FRF.reshape(1,-1)[0]
   
    peak_list = np.zeros(num_peak)
    peak_freq_list = np.zeros(num_peak)
    peak_energy_list = np.zeros(num_peak)
    kernel = np.zeros(len(frf_freq_arr))
    kernel_current_best = np.zeros(len(frf_freq_arr))
    
    for i_peak in range(num_peak):
        energy_remain_arr = np.ones(len(natrual_freq_arr))
 
        for i_freq_n in range(len(natrual_freq_arr)):#
            freq_target = natrual_freq_arr[i_freq_n]
            ampl = np.interp(freq_target, frf_freq_arr, targer_FRF)
            kernel = np.maximum(kernel_current_best, ampl*fit_gaussian_kernel(frf_freq_arr,freq_target,zeta))

            signal_remain = targer_FRF - kernel
            energy_remain_arr[i_freq_n] = simps(signal_remain**2, frf_freq_arr)

        peak_energy_list[i_peak] = np.min(energy_remain_arr)
        peak_pos = np.argmin(energy_remain_arr)
        peak_list[i_peak] =  peak_pos
        peak_freq_list[i_peak]  = natrual_freq_arr[peak_pos]
        
        freq_current_best = natrual_freq_arr[peak_pos]
        ampl_current_best= np.interp(freq_current_best, frf_freq_arr, targer_FRF)
        kernel_current_best = np.maximum(kernel_current_best, ampl_current_best*fit_gaussian_kernel(frf_freq_arr,freq_current_best,zeta))
    
    
    return np.sort(peak_list),np.sort(peak_freq_list)

def extract_mode_shape_vector(f_n_arr, frf_arr):
    ## Get mode shape from FRF
    i_model_1OG = np.array([ frf_arr['disp_ch9']['imag'].reshape(1,-1),
                            frf_arr['disp_ch10']['imag'].reshape(1,-1),
                            frf_arr['disp_ch11']['imag'].reshape(1,-1),
                            frf_arr['disp_ch12']['imag'].reshape(1,-1)])
    i_model_1OG_mat = np.vstack(i_model_1OG)
    max_i_1 = np.max(abs(i_model_1OG_mat))
    i_model_1OG_norm = i_model_1OG_mat/max_i_1

    i_model_2OG = np.array([ frf_arr['disp_ch3']['imag'].reshape(1,-1),
                            frf_arr['disp_ch13']['imag'].reshape(1,-1),
                            frf_arr['disp_ch14']['imag'].reshape(1,-1),
                            frf_arr['disp_ch15']['imag'].reshape(1,-1)])
    i_model_2OG_mat = np.vstack(i_model_2OG)
    max_i_2 = np.max(abs(i_model_2OG_mat))
    i_model_2OG_norm = i_model_2OG_mat/max_i_2

    f = frf_arr['disp_ch9']['freq'].reshape(1,-1)
    mode_shape_vector = np.zeros([len(f_n_arr),8])
    mode_freq_vector = np.zeros([len(f_n_arr)])

    i_mode = 0
    ampl_ratio = max_i_1/max_i_2
    for f_i_mode in f_n_arr:
        i_shape = 0
        mode_freq_vector[i_mode] = f_i_mode
        for i_mode_shape in range(4):
            mode_shape_1OG = np.interp(f_i_mode, f[0], i_model_1OG_norm[i_mode_shape,:])
            mode_shape_vector[i_mode,i_shape] = mode_shape_1OG
            i_shape = i_shape +1
            
        for i_mode_shape in range(4):
            mode_shape_2OG = np.interp(f_i_mode, f[0], i_model_2OG_norm[i_mode_shape,:]*ampl_ratio)
            mode_shape_vector[i_mode,i_shape] = mode_shape_2OG 
            i_shape = i_shape +1
        
        i_mode = i_mode +1

    return mode_freq_vector, mode_shape_vector

def mean_value_filted(data,std_dev_thres = 1):
    # Calculate the mean and standard deviation of the data
    mean = np.mean(data)
    std_dev = np.std(data)
    # Filter out outliers
    filtered_data = data[np.abs(data - mean) <= std_dev_thres * std_dev]
    # Calculate the mean of the remaining data
    return np.mean(filtered_data)

## MMI :Improved finite element model updating of a full-scale steel bridge using sensitivity analysis　(Bjørn T. Svendsen, 2021)

def get_MMI(f_n_model,f_n_exp, mac, f_n_ratio=0.5):
    return(1-f_n_ratio)*mac - f_n_ratio* abs(f_n_exp-f_n_model)/f_n_exp


## Obtain the exp data
# Case 13
i_file = 13
directory = r"D:/MDSI_project/MATLAB/Surrogate_main/FRF"
filename = f"mode_shape_test_{i_file}.mat"
full_path = os.path.join(directory, filename)

mode_shape_exp = sio.loadmat(full_path)
mode_shape_vector_exp = mode_shape_exp['phi']
mean_fn_1_exp = np.mean(mode_shape_vector_exp[::2,0])
mean_fn_2_exp = np.mean(mode_shape_vector_exp[1::2,0])
mode_1_exp = mode_shape_vector_exp[0::2,2]
mode_2_exp = mode_shape_vector_exp[1::2,2]

print(f"1st natural freqeuncy: {mean_fn_1_exp}, 2nd natural freqeuncy: {mean_fn_2_exp}")


1st natural freqeuncy: 12.1344, 2nd natural freqeuncy: 18.8672


In [2]:
simu_tk = simulation_PyAPDL(nproc=4,nerr=10000)
simu_tk.launch_engine()
simu_tk.mapdl.clear()

PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
Launch Pymapdl
     Launch Pymapdl successfully, duration 7.1392 seconds.


In [6]:
simu_tk.mapdl.clear()
# Initialized the parameter
# numVar = 6*4 + 1 (dr) + 3 (soil) + 1 (height) = 29
# Parameter list    = [slab ,corridor, ground, edge_wall, inner_wall, stair ]
t_para   = np.array([ 0.468 , 0.8    , 0.05  , 0.2      , 0.2       , 0.468])
e_para   = np.array([ 20e9  , 20e9   , 20e9  , 20e9     , 20e9      , 20e9])
nu_para  = np.array([ 0.27  , 0.27   , 0.27  , 0.27     , 0.27      , 0.27])
rho_para = np.array([ 2300  , 2300   , 2300  , 2300     , 2300      , 2300])

# Parameter list    = [ vs,   rho,   nu ]
soil_para = np.array([ 450, 2.3e3, 0.33])

# Damping ratio
D_r = 0.05
Height = 4
N_modes_extracted = 30



## Start the simulation
simu_tk.setting_parameter(      
                bool_SSI=True, 
                height= Height,     
                t_arr=t_para,
                e_arr=e_para,
                nu_arr=nu_para,
                rho_arr=rho_para,
                soil_arr=soil_para)
## Build the model
simu_tk.build_model()

# Excitation point: Hammer 2.0 Pos1 (shaker)
tol = 0.3
simu_tk.mapdl.nsel('S','LOC','Z',simu_tk.height*2)
simu_tk.mapdl.nsel('R','LOC','X',9.3-tol,9.3+tol)
nsel_id_1 = simu_tk.mapdl.nsel('R','LOC','Y',5.9-tol,5.9+tol)
simu_tk.mapdl.nsel('ALL')

# Apply the force on selected node and solve 
simu_tk.solve_model(Solu_type=6, N_modes = 30, Freq_Incr=1.0, End_Freq=50,Damping_ratio= D_r, excitation_node=nsel_id_1[1])

## Get natrual frequency
natrual_freq = []
for i in range(N_modes_extracted):
    modal_freq = simu_tk.mapdl.get('mode_num','MODE',i+1,'FREQ')
    natrual_freq.append(modal_freq)

## Get FRF at target point
meas_FRFs = simu_tk.get_FRFs_meas_EXAMPLE()



 Setting information
     Building inner wall           :  True
     Building stair                :  True
     Building SSI                  :  True
     Element size                  :  0.4
     Height of building            :  4.0
 Assigning parameter
     Rewrite thickness by input
     Rewrite Young's modulus by input
     Rewrite Poisson's ratio by input
     Rewrite Density by input
     Rewrite soil properties by input
 /PREP7 activated
     /PREP7 finished, duration 6.4235 seconds.
Solve model: /SOLU
 Setting information
     Solution type                 :  6
     Start frequency               :  0.001
     End frequency                 :  50
     Freqency increment            :  1.0
     Number of modes (solu_type=2) :  30
     Damping ratio                 :  0.05
     Excitation node               :  5251
Using Modal-superpostion method
     /SOLU finished, duration 27.9952 seconds.
 /POST26 actiavted
     /POST26 finished, duration 6.5765 seconds.
 Clear all the database


In [13]:
print(natrual_freq)

mode_freq_vector, mode_shape_vector = extract_mode_shape_vector(natrual_freq, meas_FRFs)

mode_freq_1_list = np.zeros(8)
mode_freq_2_list = np.zeros(8)
mac_1_list = np.zeros(8)
mac_2_list = np.zeros(8)

[index_1, index_2], [mode_freq_1_list[0],mode_freq_2_list[0]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch9']['freq'], abs( meas_FRFs['disp_ch9']['ampl']),0.05)
mac_1_list[0] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
mac_2_list[0] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

[index_1, index_2], [mode_freq_1_list[1],mode_freq_2_list[1]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch10']['freq'],abs(meas_FRFs['disp_ch10']['ampl']),0.05)
mac_1_list[1] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
mac_2_list[1] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

[index_1, index_2], [mode_freq_1_list[2],mode_freq_2_list[2]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch11']['freq'],abs(meas_FRFs['disp_ch11']['ampl']),0.05)
mac_1_list[2] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
mac_2_list[2] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)


[index_1, index_2], [mode_freq_1_list[3],mode_freq_2_list[3]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch12']['freq'],abs(meas_FRFs['disp_ch12']['ampl']),0.05)
mac_1_list[3] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
mac_2_list[3] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

[index_1, index_2], [mode_freq_1_list[4],mode_freq_2_list[4]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch3']['freq'], abs( meas_FRFs['disp_ch3']['ampl']),0.05)
mac_1_list[4] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
mac_2_list[4] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

[index_1, index_2], [mode_freq_1_list[5],mode_freq_2_list[5]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch13']['freq'],abs(meas_FRFs['disp_ch13']['ampl']),0.05)
mac_1_list[5] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
mac_2_list[5] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

[index_1, index_2], [mode_freq_1_list[6],mode_freq_2_list[6]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch14']['freq'],abs(meas_FRFs['disp_ch14']['ampl']),0.05)
mac_1_list[6] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
mac_2_list[6] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

[index_1, index_2], [mode_freq_1_list[7],mode_freq_2_list[7]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch15']['freq'],abs(meas_FRFs['disp_ch15']['ampl']),0.05)
mac_1_list[7] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
mac_2_list[7] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

f_n_1_mean = mean_value_filted(mode_freq_1_list,1)
f_n_2_mean = mean_value_filted(mode_freq_2_list,1)
mac_1_mean = mean_value_filted(mac_1_list,1)
mac_2_mean = mean_value_filted(mac_2_list,1)
print(f_n_1_mean,mac_1_mean)
print(f_n_2_mean,mac_2_mean)

print(get_MMI(f_n_1_mean,mean_fn_1_exp,mac_1_mean,0.5))
print(get_MMI(f_n_2_mean,mean_fn_2_exp,mac_2_mean,0.5))

print(mac_2_list)
print(mode_freq_2_list)



[4.28658062, 6.69074588, 10.4653121, 13.1930132, 14.0216707, 14.4440905, 14.7032602, 15.2298348, 16.0198488, 18.6019406, 20.4576139, 20.5333604, 21.9411362, 22.4178146, 24.0794943, 26.2782482, 27.443486, 27.6570976, 27.8712456, 28.1609494, 28.805114, 29.6479844, 30.3556137, 30.5550266, 30.8670426, 31.3102308, 31.7440558, 33.409178, 33.5914914, 34.4188711]
13.1930132 0.5445090635803977
15.004159971428574 0.4552154184357164
0.22863419621530434
0.12523321728022502
[0.44913672 0.44913672 0.44913672 0.00228788 0.45977444 0.45977444
 0.45977444 0.45977444]
[14.7032602 14.7032602 14.7032602 14.0216707 15.2298348 15.2298348
 15.2298348 15.2298348]


In [3]:
def object_function(
        p_0,
        p_1,
        p_2,
        p_3,
        p_4,
        p_5,
        p_6,
        p_7,
        p_8,
        p_9,
        p_10,
        p_11,
        p_12,
        p_13,
        p_14,
        p_15,
        p_16,
        p_17,
        p_18,
        p_19,
        p_20,
        p_21):
    ## Obtain the exp data
    # Case 13
    i_file = 13
    directory = r"D:/MDSI_project/MATLAB/Surrogate_main/FRF"
    filename = f"mode_shape_test_{i_file}.mat"
    full_path = os.path.join(directory, filename)

    mode_shape_exp = sio.loadmat(full_path)
    mode_shape_vector_exp = mode_shape_exp['phi']
    mean_fn_1_exp = np.mean(mode_shape_vector_exp[::2,0])
    mean_fn_2_exp = np.mean(mode_shape_vector_exp[1::2,0])
    mode_1_exp = mode_shape_vector_exp[0::2,2]
    mode_2_exp = mode_shape_vector_exp[1::2,2]

    simu_tk.mapdl.clear()
    # Initialized the parameter
    # numVar = 6*4 + 1 (dr) + 3 (soil) + 1 (height) = 29
    # Parameter list    = [slab ,corridor, ground, edge_wall, inner_wall, stair ]
    t_para   = np.array([ p_0,p_1,p_2,p_3,p_4,p_5])
    e_para   = np.array([ p_6,p_7,p_8,p_9,p_10,p_11])
    nu_para  = np.array([ 0.27  , 0.27   , 0.27  , 0.27     , 0.27      , 0.27])
    rho_para = np.array([ p_12,p_13,p_14,p_15,p_16,p_17])

    # Parameter list    = [ vs,   rho,   nu ]
    soil_para = np.array([ p_18, p_19, 0.33])

    # Damping ratio
    D_r = p_20
    Height = p_21
    N_modes_extracted = 30


    ## Start the simulation
    simu_tk.setting_parameter(      
                    bool_SSI=True, 
                    height= Height,     
                    t_arr=t_para,
                    e_arr=e_para,
                    nu_arr=nu_para,
                    rho_arr=rho_para,
                    soil_arr=soil_para)
    ## Build the model
    simu_tk.build_model()

    # Excitation point: Hammer 2.0 Pos1 (shaker)
    tol = 0.3
    simu_tk.mapdl.nsel('S','LOC','Z',simu_tk.height*2)
    simu_tk.mapdl.nsel('R','LOC','X',9.3-tol,9.3+tol)
    nsel_id_1 = simu_tk.mapdl.nsel('R','LOC','Y',5.9-tol,5.9+tol)
    simu_tk.mapdl.nsel('ALL')

    # Apply the force on selected node and solve 
    simu_tk.solve_model(Solu_type=6, N_modes = 40, Damping_ratio= D_r, excitation_node=nsel_id_1[1])

    ## Get natrual frequency
    natrual_freq = []
    for i in range(N_modes_extracted):
        modal_freq = simu_tk.mapdl.get('mode_num','MODE',i+1,'FREQ')
        natrual_freq.append(modal_freq)

    ## Get FRF at target point
    meas_FRFs = simu_tk.get_FRFs_meas_EXAMPLE()


    ## analysis
    mode_freq_vector, mode_shape_vector = extract_mode_shape_vector(natrual_freq, meas_FRFs)

    mode_freq_1_list = np.zeros(8)
    mode_freq_2_list = np.zeros(8)
    mac_1_list = np.zeros(8)
    mac_2_list = np.zeros(8)

    [index_1, index_2], [mode_freq_1_list[0],mode_freq_2_list[0]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch9']['freq'], abs( meas_FRFs['disp_ch9']['imag']),0.05)
    mac_1_list[0] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
    mac_2_list[0] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

    [index_1, index_2], [mode_freq_1_list[1],mode_freq_2_list[1]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch10']['freq'],abs(meas_FRFs['disp_ch10']['imag']),0.05)
    mac_1_list[1] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
    mac_2_list[1] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

    [index_1, index_2], [mode_freq_1_list[2],mode_freq_2_list[2]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch11']['freq'],abs(meas_FRFs['disp_ch11']['imag']),0.05)
    mac_1_list[2] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
    mac_2_list[2] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

    [index_1, index_2], [mode_freq_1_list[3],mode_freq_2_list[3]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch12']['freq'],abs(meas_FRFs['disp_ch12']['imag']),0.05)
    mac_1_list[3] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
    mac_2_list[3] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

    [index_1, index_2], [mode_freq_1_list[4],mode_freq_2_list[4]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch3']['freq'], abs( meas_FRFs['disp_ch3']['imag']),0.05)
    mac_1_list[4] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
    mac_2_list[4] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

    [index_1, index_2], [mode_freq_1_list[5],mode_freq_2_list[5]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch13']['freq'],abs(meas_FRFs['disp_ch13']['imag']),0.05)
    mac_1_list[5] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
    mac_2_list[5] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

    [index_1, index_2], [mode_freq_1_list[6],mode_freq_2_list[6]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch14']['freq'],abs(meas_FRFs['disp_ch14']['imag']),0.05)
    mac_1_list[6] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
    mac_2_list[6] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

    [index_1, index_2], [mode_freq_1_list[7],mode_freq_2_list[7]] = find_peaks_SDOFsup(2,mode_freq_vector,meas_FRFs['disp_ch15']['freq'],abs(meas_FRFs['disp_ch15']['imag']),0.05)
    mac_1_list[7] = get_MAC(mode_shape_vector[int(index_1),:],mode_1_exp)
    mac_2_list[7] = get_MAC(mode_shape_vector[int(index_2),:],mode_2_exp)

    f_n_1_mean = mean_value_filted(mode_freq_1_list,1)
    f_n_2_mean = mean_value_filted(mode_freq_2_list,1)
    mac_1_mean = mean_value_filted(mac_1_list,1)
    mac_2_mean = mean_value_filted(mac_2_list,1)

    return get_MMI(f_n_1_mean,mean_fn_1_exp,mac_1_mean,0.5), get_MMI(f_n_2_mean,mean_fn_2_exp,mac_2_mean,0.5)


In [4]:
print(object_function(0.468 , 0.8    , 0.05  , 0.2      , 0.2       , 0.468,
                      20e9  , 20e9   , 20e9  , 20e9     , 20e9      , 20e9,
                      2300  , 2300   , 2300  , 2300     , 2300      , 2300,
                      450, 2.3e3, 0.05, 4 ))

 Setting information
     Building inner wall           :  True
     Building stair                :  True
     Building SSI                  :  True
     Element size                  :  0.4
     Height of building            :  4.0
 Assigning parameter
     Rewrite thickness by input
     Rewrite Young's modulus by input
     Rewrite Poisson's ratio by input
     Rewrite Density by input
     Rewrite soil properties by input
 /PREP7 activated
     /PREP7 finished, duration 6.5520 seconds.
Solve model: /SOLU
 Setting information
     Solution type                 :  6
     Start frequency               :  0.001
     End frequency                 :  50
     Freqency increment            :  0.5
     Number of modes (solu_type=2) :  40
     Damping ratio                 :  0.05
     Excitation node               :  5251
Using Modal-superpostion method
     /SOLU finished, duration 52.1722 seconds.
 /POST26 actiavted
     /POST26 finished, duration 7.7616 seconds.
 Clear all the database


In [1]:
import numpy as np

# Define the model
def model(a, b):
    # Example model output, can be replaced by a more complex function
    return a * np.exp(-b) + b**2

# Initial parameters and their baseline values
params = {'a': 1.0, 'b': 2.0}
baseline_output = model(params['a'], params['b'])

# Perturbation size (e.g., 1% of the parameter value)
perturbation_size = 0.01

# Dictionary to store sensitivities
sensitivities = {}

for param_name in params:
    # Calculate the perturbation for the current parameter
    perturbation = params[param_name] * perturbation_size

    # Positive perturbation
    params[param_name] += perturbation
    output_pos = model(**params)

    # Negative perturbation
    params[param_name] -= 2 * perturbation  # revert and then apply negative perturbation
    output_neg = model(**params)

    # Revert parameter to original value
    params[param_name] += perturbation

    # Calculate sensitivity (relative sensitivity in this case)
    sensitivity = ((output_pos - output_neg) / (2 * perturbation)) / baseline_output
    sensitivities[param_name] = sensitivity

print("Sensitivity of each parameter:", sensitivities)


Sensitivity of each parameter: {'a': 0.03272655636539199, 'b': 0.9345447054551705}
